# Classes and Objects: *Part III*

> <font color='green'>CS196 - Lecture 4</font>
>
> **Instructor:** *Dr. V*

---

----
### Review

Dunder methods (double-underscore methods; a.k.a. magic methods) are a way for you to tie your custom classes to common python functionality.

Defining dunder methods for your custom classes enables class instances to
- have custom initialization, representation, and stringification
- work with common python functions and operators
- have custom access to object attributes
- have custom dictionary-like functionality where values can be accessed by keys, indices, or slices

Private method/attribute names should start with a single underscore (not double-underscore).

Some common dunder methods:
- `__init__` -- called when your object is first being created
- `__str__` -- called whenever your object is being converted to a string
- `__eq__` -- called when your object is being compared to another using the `==` operator
- `__lt__` -- called when your object is being compared to another using the `<` operator
- `__gt__` -- called when your object is being compared to another using the `>` operator
- `__add__` -- called when your object is being added to another using the `+` operator
- `__sub__` -- called when your object is being subtracted from another using the `-` operator

The only way you can save and access some value in an instance is by using `self.` followed by attribute name.
- any other variables will be forgotten once a method finishes execution

In [ ]:
class Dog:
    def __init__(self,name,age):
        self.age = age
    def __str__(self):
        return f"Dog by the name of {name} is {age} years old."

# what does this print?
dog = Dog('sparky',7)
print(dog)

NameError: ignored

How do we fix the code above?

----
### `@dataclass`

There is a decorator in python called `@dataclass` that can make class creation much easier.

Normally, when you define a class, you'll want to instantiate a whole bunch of instance attributes inside the `__init__`, as such:

In [ ]:
class Person:
    def __init__(self,name,age,addr):
        self.name = name
        self.age = age
        self.addr = addr


You'll probably also want to add `__str__` and `__repr__` dunder methods, so that you can print and examine your objects, as such:

In [ ]:
class Person:
    def __init__(self,name,age,addr):
        self.name = name
        self.age = age
        self.addr = addr
    def __str__(self):
        return f'Person (name={self.name}, age={self.age}, addr={self.addr})'
    def __repr__(self):
        return f'Person (name={self.name}, age={self.age}, addr={self.addr})'

p1 = Person('jenn',23,'1234 5th ave')
print(p1)

Person (name=jenn, age=23, addr=1234 5th ave)


You'll probably also want to add all the comparison methods to compare your instances, so that your objects can be compared to each other, and potentially ordered and sorted:
- `__eq__`
- `__lt__`
- `__gt__`
- `__le__`
- `__ge__`

In [ ]:
class Person:
    def __init__(self,name,age,addr):
        self.name = name
        self.age = age
        self.addr = addr
    def __str__(self):
        return f'Person (name={self.name}, age={self.age}, addr={self.addr})'
    def __repr__(self):
        return f'Person (name={self.name}, age={self.age}, addr={self.addr})'
    def __eq__(self, x):
        return (self.name,self.age,self.addr) == x
    def __lt__(self, x):
        return (self.name,self.age,self.addr) < x
    def __gt__(self, x):
        return (self.name,self.age,self.addr) > x
    def __le__(self, x):
        return (self.name,self.age,self.addr) <= x
    def __ge__(self, x):
        return (self.name,self.age,self.addr) >= x

p1 = Person('jenn',23,'1234 5th ave')
p2 = Person('jenn',23,'1234 5th ave')
p3 = Person('joe',21,'2345 6th ave')
print( p1 == p2 )
print( p1 <= p2 )

True
True


Ugh... 😒

Creating all these methods every time you create a class can get really annoying, especially if you have lots of object attributes.

However, python's `@dataclass` can automatically create all those methods for you!

In [ ]:
from dataclasses import dataclass

@dataclass
class Person:
    name: str
    age: int
    addr: str

p1 = Person('jenn',23,'1234 5th ave')
p2 = Person('jenn',23,'1234 5th ave')
print(p1)
print(p1 == p2)

Person(name='jenn', age=23, addr='1234 5th ave')
True


By default `@dataclass` decorator will add `__init__`, `__str__`, `__repr__`, and `__eq__` methods for you, but not `__lt__`, `__gt__`, `__le__`, or `__ge__`.

To make sure `@dataclass` adds the `__lt__`, `__gt__`, `__le__`, and `__ge__` methods, use `@dataclass(order=True)`, as such:

In [ ]:
from dataclasses import dataclass

@dataclass(order=True)
class Person:
    name: str
    age: int
    addr: str

p1 = Person('jenn',23,'1234 5th ave')
p2 = Person('joe',21,'2345 6th ave')
print(p1)
print(p1 == p2)
print(p1 < p2)

Person(name='jenn', age=23, addr='1234 5th ave')
False
True


In addition to all the methods a dataclass will generate for you, you can of course add your own custom methods.

In this way, a `@dataclass` decorator will enable you to automatically create all the common methods that you don't want to be writing yourself, and allow you to get to writing the custom methods.

In [ ]:
from dataclasses import dataclass

@dataclass(order=True)
class Person:
    name: str
    age: int
    addr: str
    def walk(self):
        print(f'{self.name} walks')

p1 = Person('jenn',23,'1234 5th ave')
p1.walk()

jenn walks


But wait -- if `@dataclass` automatically creates the `__init__` method, how can I add custom functionality upon initialization?

For example, let's say I wanted to capitalize `name` or calculate the person's birth year based on their `age`?

`@dataclass` instances will call a special dunder method called `__post_init__` right after `__init__` method is done.

In [ ]:
from dataclasses import dataclass
from datetime import date

@dataclass(order=True)
class Person:
    name: str
    age: int
    addr: str
    def __post_init__(self):
        assert isinstance(self.age, (float, int)), "Age must be a number."
        self.name = self.name.upper()
        self.birthYear = date.today().year - int(self.age)
        print(f'{self.name} was born in {self.birthYear}')

# what does this print?
p1 = Person('jenn',23,'1234 5th ave')


JENN was born in 2000


You can also specify defaults for your instance attributes in `@dataclass` --

In [1]:
from dataclasses import dataclass

@dataclass(order=True)
class Person:
    name: str
    age: int = 20      # age will have a default value of 20
    addr: str = None   # addr will have a default value of None

p1 = Person('jenn',23)
print(p1)

Person(name='jenn', age=23, addr=None)


You can even specify functions that will automatically generate defaults for you.

For example, you can specify `list` as a function to create a new list as a default value whenever your object attribute is initialized.

Or it can be some custom function.

To do this, import `field` from dataclasses, and then use `field(default_factory = `*functionName*`)` to specify a function as a default value.

In [ ]:
from dataclasses import dataclass, field
import random

def createStudentId() -> str:
    '''Returns a string of 8 random digits'''
    return "".join(random.choices('0123456789',k=8))

@dataclass(order=True)
class Student:
    name: str
    age: int
    id: str = field(default_factory=createStudentId)
    courses: list = field(default_factory=list)

# what does this print?
p1 = Student('jenn',23)
print(p1)

Student(name='jenn', age=23, id='11845756', courses=[])


In [ ]:
# what does this print?
p1 = Student('jenn',23,'12345678',['CS196','CS242'])
print(p1)

Student(name='jenn', age=23, id='12345678', courses=['CS196', 'CS242'])


What if you wanted the `Student` id attribute to never be specified during initialization, you wanted it to *always* get assigned to some random string produced by the `createStudentId` function?

In [ ]:
@dataclass(order=True)
class Student:
    id: str = field(default_factory=createStudentId, init=False)  # id isn't part of __init__ args
    name: str
    age: int
    courses: list = field(default_factory=list)

# what does this print?
p1 = Student('jenn',23,['CS196','CS242'])
print(str(p1))

Student(id='13753194', name='jenn', age=23, courses=['CS196', 'CS242'])


The `field` function also allows you to control whether your attribute gets returned as part of the `__repr__` and `__str__` strings by using the `repr` flag --

In [ ]:
@dataclass(order=True)
class Student:
    id: str = field(default_factory=createStudentId, init=False)  # id isn't part of __init__ args
    name: str
    age: int
    courses: list = field(default_factory=list, repr=False)       # courses isn't part of __repr__

# what does this print?
p1 = Student('jenn',23,['CS196','CS242'])
print(p1)

Student(id='68925855', name='jenn', age=23)


Note that if you are trying to order/sort your dataclass instances, they will be sorted by the first specified attribute, then by the second, and so on.

So, in the example above, students would be sorted by `id` because `id` was the first attribute listed in dataclass definition.

If you wanted to change this behavior, you could change the order of attribute specification.

Alternatively, you can add an additional attribute for indexing as your first attribute, as such --

In [ ]:
@dataclass(order=True)
class Person:
    indx: int = field(init=False,repr=False)
    name: str
    age: int
    def __post_init__(self):
        self.indx = self.age

people = [ Person('jenn',23), Person('joe',20), Person('alicia',21), Person('yvonne',17) ]

# what does this print?
print( sorted( people ) )

[Person(name='yvonne', age=17), Person(name='joe', age=20), Person(name='alicia', age=21), Person(name='jenn', age=23)]


You can do some other neat stuff with dataclasses and fields, e.g.:
- make attributes unchangeable by specifying that the dataclass is frozen (`@dataclass(frozen=True)`)
- make attribute access about 20% faster (`@dataclass(slots=True)`)

This a good 20m video on dataclasses that goes into more detail:

https://www.youtube.com/watch?v=CvQ7e6yUtnw

----
### Named Tuples

Instead of defining a class, there are many times when it would suffice to use a `namedtuple`.

A named tuple allows you to specify attribute names for each of the tuple's slots.

In the example below you see a named tuple called `Person` with 3 slots -- `name`, `age`, and `addr`.

In this way, any instance of `Person`, `p`, would allow you to get values for `p.name`, `p.age`, and `p.addr`.

In [ ]:
from collections import namedtuple

Person = namedtuple('Person',('name','age','addr'))

p1=Person('jenn',23,'1234 5th ave')
p2=Person(name='bob',addr='4321 2nd ave',age=21) # you can use keyword args to create a namedtuple instance

# what do you think this prints?
print(p1)
print(p2)
print(p1.age)
print(p2.age)

Person(name='jenn', age=23, addr='1234 5th ave')
Person(name='bob', age=21, addr='4321 2nd ave')
23
21


So what's the difference?

Why define a `class` or `@dataclass` instead of `namedtuple`?

----
### Instance vs Class attributes

Not only can you define functions (i.e., methods) at the top level of your class definition block, you can also define variables (attributes).

Variables defined in such manner are **not** instance attributes, they are class attributes.

In the example below, the variable `wheels` belongs to the class `Car`, not to specific instances of that class.

On the other hand, the variable `self.name` belongs to each individual instance of `Car`.

In [ ]:
class Car:
    wheels = 4    # <- Class attribute
    def __init__(self, name):
        self.name = name    # <- Instance attribute

car1 = Car('subaru')
car2 = Car('jeep')

# what do you think this prints?
print(car1.name, car1.wheels)
print(car2.name, car2.wheels)

subaru 4
jeep 4


In [ ]:
# what do you think this prints?
print(Car.wheels)

# what do you think this prints?
print(Car.name)

4


AttributeError: ignored

In [ ]:
Car.wheels = 5

# what do you think this prints?
print(car1.name, car1.wheels)
print(car2.name, car2.wheels)

subaru 5
jeep 5


In [ ]:
Car.wheels = 4
car2.wheels = 5   # declaring an instance attribute called wheels

# what do you think this prints?
print(car1.name, car1.wheels)
print(car2.name, car2.wheels)

subaru 4
jeep 5


In [ ]:
# what do you think this prints?
print(car2.wheels)
print(car2.__class__.wheels)

5
4


You can treat class attribute values as **defaults** for any object that is an instance of that class (as in the examples above).

However, you can also treat them as
- constant values for all objects in the class
- variables to keep track of things having to do with all instances of the class

In [ ]:
class Car:
    WHEELS = 4              # <- Class constant
    count = 0               # <- Class attribute
    def __init__(self, name):
        self.name = name    # <- Instance attribute
        self.__class__.count+=1

car1 = Car('subaru')
car2 = Car('jeep')
car3 = Car('tesla')

# what do you think this prints?
print(car1.WHEELS)
print(Car.count)

4
3


----
### Class and Object dunder attributes

We may recall that objects have dunder attributes `__class__` and `__dict__`.

Classes have several dunder attributes, as well:
- `__dict__`: stores class attributes and other definitions
- `__name__`: stores class name
- `__module__`: stores the name of the module where this class was defined
- `__bases__`: stores class base classes (more on this in the next lecture)
- `__mro__`: stores class method resolution order (more on this in the next lecture)


In [ ]:
# what do you think this prints?
print( Car.__name__ )

Car


In [ ]:
# what do you think this prints?
C = Car
print( C.__name__ )

Car


However, unlike other class attributes, dunder attributes are not accessible from objects.

In [ ]:
print(car1.__name__)

AttributeError: ignored

To access class dunder attributes from objects, use `.__class__` to access the class first:

In [ ]:
print(car1.__class__.__name__)

There is also a class dunder *method* called `__new__`, which executes prior to instance method `__init__`.

We'll talk more about this next time.

----
### Instance vs Class methods

Just like classes can have their own attributes, they can have their own methods.

Class methods are differentiated from instance methods by using the `@classmethod` decorator just above the definition.

The first argument for every class method should be `cls` (as opposed to `self` which is the first argument for instance methods).

In [ ]:
class Car:
    wheels = 4                 # class attribute

    def __init__(self, name):  # instance method
        self.name = name       # instance attribute
    
    @classmethod
    def go(cls):               # class method
        print(f'car drives on {cls.wheels} wheels')

    def stop(self):            # instance method
        print(f'{self.name} stops')

car1 = Car('subaru')

In [ ]:
# what do you think this prints?
car1.go()
Car.go()

In [ ]:
# what do you think this prints?
car1.stop()
Car.stop()

In python you cannot overload methods; i.e., 
- you **cannot** have two methods with the same name (e.g., one class method and one instance method).

In [ ]:
class Car:
    wheels = 4                 # class attribute

    def __init__(self, name):  # instance method
        self.name = name       # instance attribute

    @classmethod
    def go(cls):               # class method
        print(f'car drives on {cls.wheels} wheels')

    def go(self):              # instance method will overwrite the classmethod above
        print(f'{self.name} drives on {self.wheels} wheels')

car1 = Car('subaru')

# what do you think this prints?
car1.go()
Car.go()

Just like any function, class methods can have multiple arguments (in addition to `cls`).

In [ ]:
class Car:
    wheels = 4                 # class attribute

    @classmethod
    def go(cls, speed):        # class method
        print(f'car drives on {cls.wheels} wheels {speed}')

Car.go('fast')

----
### Static methods

Just like `@classmethod`, a static method (marked by `@staticmethod`) is also a method that belongs to the class, rather than to an instance of a class.

Unlike `@classmethod`, a static method does not require `cls` as the first argument.

In [ ]:
class Car:
    wheels = 4                 # class attribute

    def __init__(self, name):  # instance method
        self.name = name       # instance attribute
    
    @classmethod
    def go(cls):               # class method
        print(f'car drives on {cls.wheels} wheels')

    @staticmethod
    def stop():                # static method
        print(f"car stops")

Car.stop()

So what's the point of `@classmethod` then?

Why would we ever want to use `@classmethod` when `@staticmethod` also belongs to a class, and doesn't even require the extra `cls` argument?

🤔

Just like any functions, static methods can have multiple arguments.

In [ ]:
class Math:
    @staticmethod
    def add(x,y):
        return x+y

    @staticmethod
    def pow(x,y):
        return x**y

# what do you think this prints?
print( Math.add(3,4) )
print( Math.pow(3,2) )

7
9


Note that the code above has no instance methods or attributes.

In this way, the `Math` class definition becomes a *namespace* --
- instead of having global functions `add` and `pow` we have `Math.add` and `Math.pow`

This may be something you'll want as a part of your code (though it's often better to just create a separate module for this purpose).

In [ ]:
class Math:
    PI = 3.14159

    @staticmethod
    def add(x,y):
        return x+y

    @staticmethod
    def pow(x,y):
        return x**y
    
    @classmethod
    def areaOfCircle(cls, r):
        return cls.PI * cls.pow(r,2)


# what do you think this prints?
print( Math.add(3,4) )
print( Math.pow(3,2) )
print( Math.PI )
print( Math.areaOfCircle(1) )

Why did I define `areaOfCircle` as a class method, whereas `add` and `pow` are static methods?

----
### Inner Classes


In [ ]:
from dataclasses import dataclass, field

class Students:
    _students = []
    @classmethod
    def getNextStudentId(cls):
        return f'{len(cls._students):04}'
    @classmethod
    def add(cls, student):
        cls._students.append(student)
    @classmethod
    def get(cls, id):
        return cls._students[int(id)]
    @classmethod
    def showAll(cls):
        for student in cls._students:
            print(student)

@dataclass
class Student:
    id: str = field(default_factory=Students.getNextStudentId, init=False)
    name: str
    age: int
    courses: list = field(default_factory=list)

Students.add( Student('jenn',18,['CS196','CS242']) )
Students.add( Student('joe',17,['CS196','CS230']) )
Students.add( Student('anna',19,['CS195','CS119','BUS110']) )

In [ ]:
# what does this print?
Students.showAll()

In [ ]:
# what does this print?
print( Students.get('0002') )

We can make the code above even cleaner, by moving the `Student` class out of the global scope, and into the `Students` name space.

We can continue to use Students as our namespace, and just move the Student class inside of that namespace.

Yes, python allows you to **define classes within other classes**.

In [ ]:
from dataclasses import dataclass, field

class Students:
    _students = []
    @classmethod
    def getNextStudentId(cls):
        return f'{len(cls._students):04}'
    @classmethod
    def add(cls, name, age, courses=None):
        cls._students.append( cls.Student(cls.getNextStudentId(), name, age, courses or []) )
    @classmethod
    def get(cls, id):
        return cls._students[int(id)]
    @classmethod
    def showAll(cls):
        for student in cls._students:
            print(student)
    @dataclass
    class Student:
        id: str
        name: str
        age: int
        courses: list

In [ ]:
Students.add( 'jenn', 18, ['CS196','CS242'] )
Students.add( 'joe', 17, ['CS196','CS230'] )
Students.add( 'anna', 19 )

Students.showAll()

A class defined within another class is called an **inner class**.

An inner class can only be accessed from within its outer class, not from the global namespace.

In [ ]:
Students

In [ ]:
Students.Student

In [ ]:
Student

----
### Getters, Setters, Deleters

In [ ]:
class Employee:
    def __init__(self, firstName, lastName):
        self.firstName = firstName
        self.lastName = lastName
        self.email = f'{self.firstName}.{self.lastName}@email.com'
        self.fullName = f'{self.firstName} {self.lastName}'

e = Employee('Jenn','Jardene')

# what does this print?
print(e.fullName, e.email)

In [ ]:
e.firstName = 'Jennifer'

# what does this print?
print(e.fullName, e.email)

The problem is that setting `.firstName` or `.lastName` does nothing to update the `.email` or `.fullName` attributes.

Python has a special decorator called `@property` that enables the creation of custom getter/setter/delete methods that could help resolve this issue.

For example, we can change the code above to turn `.email` and `.fullName` attributes into properties that get automatically generated every time they are requested --

In [ ]:
class Employee:
    def __init__(self, firstName, lastName):
        self.firstName = firstName
        self.lastName = lastName
    @property
    def email(self):
        return f'{self.firstName}.{self.lastName}@email.com'
    @property
    def fullName(self):
        return f'{self.firstName} {self.lastName}'


e = Employee('Jenn','Jardene')
e.firstName = 'Jennifer'

# what does this print?
print(e.fullName, e.email)

However, the code above would not allow us to change the `.email` or `.fullName` attributes, only to get them.

So, if you wanted to, for example, set the `fullName`, and automatically change `firstName` and `lastName` based on `fullName`, we would need to define a setter --

In [ ]:
class Employee:
    def __init__(self, firstName, lastName):
        self.firstName = firstName
        self.lastName = lastName
    @property
    def email(self):
        return f'{self.firstName}.{self.lastName}@email.com'
    @property
    def fullName(self):
        return f'{self.firstName} {self.lastName}'
    @fullName.setter
    def fullName(self, value):
        self.firstName, self.lastName = value.split()
    
e = Employee('Jenn','Jardene')
e.fullName = 'Jennifer Jardensky'

# what does this print?
print(e.firstName)
print(e.lastName)
print(e.fullName, e.email)

Just like we added getter and setter methods for the `fullName` property, we can also add a deleter method for it --

In [ ]:
class Employee:
    def __init__(self, firstName, lastName):
        self.firstName = firstName
        self.lastName = lastName
    @property
    def email(self):
        return f'{self.firstName}.{self.lastName}@email.com'
    @property
    def fullName(self):
        return f'{self.firstName} {self.lastName}'
    @fullName.setter
    def fullName(self, value):
        self.firstName, self.lastName = value.split()
    @fullName.deleter
    def fullName(self):
        self.firstName = None
        self.lastName = None

    
e = Employee('Jenn','Jardene')
del e.fullName

# what does this print?
print(e.firstName)
print(e.lastName)
print(e.fullName, e.email)

----
### Summary

- Python `@dataclass` decorator enables us to quickly create classes that have `__init__`, `__str__`, `__repr__`, and `__eq__` methods defined for us
  - It can even define order comparison methods for us (i.e., `__lt__`, `__gt__`, `__le__`, or `__ge__`)
- A named tuple can be another nice alternative to defining classes or dataclasses
  - However, named tuples are immutable, and have no methods
- Just like objects can have attributes and methods, the classes themselves can also have attributes and methods
  - A class attribute value may be thought of as:
    - a **default** value for all instances of that class
    - a CONSTANT for all instances of the class
    - a way to keep track of all things having to do with that class
  - There are two types of methods that belong to a class
    - `@classmethod`: has access to class attributes and other class/static methods
    - `@staticmethod`: has no access to its own class, just executes code in isolation
- Classes defined inside of other classes are called inner classes
  - An inner class is only visible from within its outer class, not globally
- Python enables you to create getters, setters, and deleters
  - Use `@property` decorator to create a getter method
  - Use `@x.setter` decorator to create a setter for variable `x`
  - Use `@x.deleter` decorator to create a deleter for variable `x`  

----
### Assignment 3

(*due before next lecture*)

Create a Jupyter notebook called `CS196-a3.ipynb`

**DO NOT INCLUDE YOUR NAME ANYWHERE IN THIS FILE OR IN FILENAME**

In this notebook you should have the following:

1. Create some class that includes the following
    - instance attributes (at least 2)
    - instance methods (at least 2)
    - class attributes (at least 2)
    - class methods (at least 2)
    - static methods (at least 2)

2. Create a few objects of this class

3. Show off all implemented functionality

**DO NOT HAVE THE SAME CLASS DEFINITIONS AS YOUR CLASSMATES**:

- Even if you are working together with your peers, make sure that you implement different classes/attributes/methods and that they do different types of things.

Add docstrings and comments (and/or markdown) where appropriate.

Code will be evaluated for:
1. code is written and works as intended (e.g., correct calls, correct output, no errors)
2. clean/efficient code (e.g., no unnecessary code)
3. naming conventions (e.g., class names are UpperCamelCase)
4. readability (e.g., meaningful names, separation of code into separate cells)
5. documentation (e.g., docstrings, comments, argument type specification)
* click "View Rubric" on blackboard under this assignment for more details

Execute all cells in this notebook, save, and upload the notebook on blackboard.